In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random
import os
import pandas as pd
import imageio
from scipy.optimize import minimize
from joblib import Parallel, delayed
import pickle

df = pd.read_csv("df_normalizedcleanfinalscaled.csv")


,Unnamed: 0,yearid,year_1b,year_2,id_1b,id_2,coninc_1b,coninc_2,race_1b,sexnow_1b,sexnow1_2,affrmact_norm1b,gunlaw_norm1b,partyid_norm1b,affrmact_norm2,gunlaw_norm2,partyid_norm2
0,1,20180001,2018,2020,1,810,NaN,36960.0,1,NaN,1,-1.000000,1,-0.666667,-1.000000,1.0,-1.000000
1,2,20180006,2018,2020,6,815,NaN,25200.0,1,NaN,2,0.333333,1,0.333333,-0.333333,1.0,0.333333
2,3,20180011,2018,2020,11,819,112160.0,94080.0,1,NaN,1,-1.000000,1,-1.000000,-1.000000,1.0,-1.000000
3,4,20180016,2018,2020,16,822,47317.5,15960.0,1,NaN,2,-0.333333,1,0.000000,-1.000000,1.0,0.000000
4,5,20180063,2018,2020,63,836,38555.0,NaN,2,NaN,2,0.333333,1,1.000000,1.000000,1.0,1.000000


In [4]:

# Load static graph from file
with open("fixed_graph.pkl", "rb") as f:
    G = pickle.load(f)

# Visualize or verify structure
print(f"Loaded fixed graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")

opinion_cols_1b = ["affrmact_norm1b", "gunlaw_norm1b", "partyid_norm1b"]
opinion_cols_2 = ["affrmact_norm2", "gunlaw_norm2", "partyid_norm2"]
# Clean the dataset
df_clean = df.dropna(subset=opinion_cols_1b+opinion_cols_2)
#Populate intial opinions frmo data
A_initial = df_clean[opinion_cols_1b].values
A_final = df_clean[opinion_cols_2].values
N = A_initial.shape[0]
l = A_initial.shape[1]
lambda_val = 0.5

Loaded fixed graph with 281 nodes and 14191 edges


In [3]:
#computing clustering coefficient and centrality measures
clustering_coeffs = nx.clustering(G)
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)

#converting to a frame for analysis
network_metrics = pd.DataFrame({
    "Node": list(G.nodes()),
    "Degree": [G.degree(n) for n in G.nodes()],
    "Clustering Coefficient": [clustering_coeffs[n] for n in G.nodes()],
    "Degree Centrality": [degree_centrality[n] for n in G.nodes()],
    "Betweenness Centrality": [betweenness_centrality[n] for n in G.nodes()]
})

#compute average clustering based on topic alignment
topic_similarity = {"affirmative_action": [], "gun_laws": [], "party_id": []}

for i in range(N):
    for j in range(i + 1, N):
        distance = np.linalg.norm(A_initial[i] - A_initial[j])**2
        probability = np.exp(- (1 /(4*lambda_val))*distance) #prob connected
        
        #separate contributions from each topic dimension
        for idx, topic in enumerate(["affirmative_action", "gun_laws", "party_id"]):
            topic_distance = (A_initial[i, idx] - A_initial[j, idx])**2
            topic_prob = np.exp(- (1 / (4*lambda_val)) * topic_distance)
            topic_similarity[topic].append(topic_prob)

#computing average connection probability per topic
average_topic_similarity = {topic: np.mean(values) for topic, values in topic_similarity.items()}


#saving network metrics to a csv file 
network_metrics.to_csv("[INSERT DIRECTORY]/1Pnetwork_metricsparallelto-1.csv", index=False)
df_topic_similarity = pd.DataFrame.from_dict(average_topic_similarity, orient="index", columns=["Avg Probability"])
df_topic_similarity.to_csv("/[INSERT DIRECTORY]/1Ptopic_similarityparallelto-1.csv", index=True)


Network Metrics (First 5 Rows):
   Node  Degree  Clustering Coefficient  Degree Centrality  \
0     0     120                0.586695           0.427046   
1     1     127                0.523060           0.451957   
2     2      92                0.544673           0.327402   
3     3     138                0.524807           0.491103   
4     4     115                0.543555           0.409253   

   Betweenness Centrality  
0                0.002565  
1                0.003350  
2                0.001986  
3                0.004589  
4                0.002662  

Average Connection Probability Per Topic:
                    Avg Probability
affirmative_action         0.731862
gun_laws                   0.670030
party_id                   0.719685


In [5]:
#Covariance/interdependance matrix
topics2018 = opinion_cols_1b
topics2020 = opinion_cols_2
df_change = df_clean[topics2020].values - df_clean[topics2018].values
C_covar = np.cov(df_change, rowvar=False)
C_df = pd.DataFrame(C_covar, index=topics2018, columns=topics2018)
C_stochastic = C_df.div(C_df.sum(axis=1), axis=0).clip(lower=0)
C_stochastic = C_stochastic.div(C_stochastic.sum(axis=1), axis=0).values



In [6]:
#Player Class and Simulation 
class Player:
    def __init__(self, theta_low, theta_high, delta, beta, topic_index, agent_id, push_toward, l, kappa, gamma):
        self.theta_low = theta_low
        self.theta_high = theta_high
        self.delta = delta
        self.beta = beta
        self.topic_index = topic_index
        self.agent_id = agent_id
        self.push_toward = push_toward
        self.l = l
        self.kappa = kappa
        self.gamma = gamma

    def compute_w_link(self, A_updated):
        N = A_updated.shape[0]
        w_link_matrix = np.zeros((N, N))
        for i in range(N):
            for j in range(i + 1, N):
                distance = np.linalg.norm(A_updated[i] - A_updated[j])
                w_link_matrix[i, j] = np.exp(-distance)
                w_link_matrix[j, i] = w_link_matrix[i, j]
        return w_link_matrix

    def send_message_single_agent(self, network, initial_node, message, C):
        active_nodes = set()
        claimed_nodes = {}
        A_updated = np.copy(A_initial)

        alignment = self.beta * (1 - np.linalg.norm(A_initial[initial_node] - message) / (2 * np.sqrt(self.l)))

        if alignment > self.theta_low:
            w_link_matrix = self.compute_w_link(A_updated)
            neighbors = list(network.neighbors(initial_node))
            Z_i = np.sum(w_link_matrix[initial_node, neighbors])
            neighbor_influence = np.sum([A_updated[j] * w_link_matrix[initial_node, j] for j in neighbors]) / Z_i if Z_i > 0 else 0  
            A_updated[initial_node] = (1 - self.kappa - self.gamma) * A_updated[initial_node] + self.kappa * (C @ message) + self.gamma * neighbor_influence
            active_nodes.add(initial_node)
            claimed_nodes[initial_node] = self.agent_id

        timestep_count = 0
        while True:
            new_active_nodes = set()
            w_link_matrix = self.compute_w_link(A_updated)
            for node in active_nodes:
                for neighbor in network.neighbors(node):
                    if neighbor in claimed_nodes:
                        continue
                    if w_link_matrix[node, neighbor] > self.delta:
                        alignment = self.beta * (1 - np.linalg.norm(A_updated[neighbor] - message) / (2 * np.sqrt(self.l)))
                        if alignment > self.theta_low:
                            neighbors = list(network.neighbors(node))
                            Z_i = np.sum(w_link_matrix[node, neighbors])
                            neighbor_influence = np.sum([A_updated[j] * w_link_matrix[node, j] for j in neighbors]) / Z_i if Z_i > 0 else 0
                            A_updated[neighbor] = (1 - self.kappa - self.gamma) * A_updated[neighbor] + self.kappa * (C @ message) + self.gamma * neighbor_influence
                            new_active_nodes.add(neighbor)
                            claimed_nodes[neighbor] = self.agent_id
            if not new_active_nodes:
                break
            active_nodes.update(new_active_nodes)
            timestep_count += 1
        #fixing influence
        total_influence = self.push_toward*np.sum(A_updated[:, self.topic_index] - A_initial[:, self.topic_index])
        return total_influence, A_updated, claimed_nodes, timestep_count



    def optimize_strategy_single_agent(self, network, C, n_jobs=-1):
        base_vectors = [np.eye(self.l)[i] for i in range(self.l)] + [-np.eye(self.l)[i] for i in range(self.l)]
        sampled_messages = base_vectors + [np.random.uniform(-1, 1, self.l) for _ in range(3)]

        def evaluate_strategy(initial_node, message):
            influence, _, _, _ = self.send_message_single_agent(network, initial_node, message, C)
            return (initial_node, message, influence)

        all_jobs = [(node, message) for node in network.nodes for message in sampled_messages]

        results = Parallel(n_jobs=n_jobs)(
            delayed(evaluate_strategy)(node, msg) for node, msg in all_jobs
        )

        #optimizing for best 
        best_node, best_message, max_influence = max(results, key=lambda x: x[2])

        return best_node, best_message, max_influence


def simulate_diffusion(player, network, initial_node, message, C):
    active_nodes_history = []
    A_updated = np.copy(A_initial)
    active_nodes = set([initial_node])
    claimed_nodes = {initial_node: player.agent_id}
    active_nodes_history.append(set(active_nodes))

    influence_history = []
    while True:
        new_active_nodes = set()
        w_link_matrix = player.compute_w_link(A_updated)
        for node in active_nodes:
            for neighbor in network.neighbors(node):
                if neighbor in claimed_nodes:
                    continue
                if w_link_matrix[node, neighbor] > player.delta:
                    alignment = player.beta * (1 - np.linalg.norm(A_updated[neighbor] - message) / (2 * np.sqrt(player.l)))
                    if alignment > player.theta_low:
                        neighbors = list(network.neighbors(node))
                        Z_i = np.sum(w_link_matrix[node, neighbors])
                        neighbor_influence = np.sum([A_updated[j] * w_link_matrix[node, j] for j in neighbors]) / Z_i if Z_i > 0 else 0
                        A_updated[neighbor] = (1 - player.kappa - player.gamma) * A_updated[neighbor] + player.kappa * (C @ message) + player.gamma * neighbor_influence
                        new_active_nodes.add(neighbor)
                        claimed_nodes[neighbor] = player.agent_id
        if not new_active_nodes:
            break
        active_nodes.update(new_active_nodes)
        active_nodes_history.append(set(active_nodes))
        #fixing influence
        influence_t = player.push_toward*np.sum(A_updated[:, player.topic_index] - A_initial[:, player.topic_index])
        influence_history.append(influence_t)

    return active_nodes_history, A_updated, claimed_nodes, influence_history


def visualize_single_agent_diffusion(G, active_nodes_history, A_updated, claimed_nodes, topic_index, message):
    topic_names = {0: "Affirmative Action", 1: "Gun Permits", 2: "Political Party ID"}
    topic_name = topic_names.get(topic_index, "Unknown Topic")
    max_timesteps = len(active_nodes_history)
    pos = nx.spring_layout(G, seed=42)

    for t in range(max_timesteps):
        plt.figure(figsize=(8, 6))
        node_colors = []
        current_claimed = set()
        for i in range(t + 1):
            current_claimed.update(active_nodes_history[i])
        for n in G.nodes():
            node_colors.append("red" if n in current_claimed else "gray")
        nx.draw(G, pos, with_labels=False, node_color=node_colors, node_size=50, edge_color="lightgray")
        plt.title(f"{topic_name} – Agent Influence at Step {t+1}")
        filename = os.path.join("/scratch/network/re0578/sameG/1Player/to-1/networkdiffusion", f"{topic_name} – Agent(-1) Influence at Step {t+1}.png")

        plt.savefig(filename, dpi=300)
        plt.close()
        #plt.show()



In [9]:
def run_single_simulation(sim_idx, topic_index, push_toward, G, C, A_initial, l, player_config):
    player = Player(
        theta_low=player_config["theta_low"],
        theta_high=player_config["theta_high"],
        delta=player_config["delta"],
        beta=player_config["beta"],
        topic_index=topic_index,
        agent_id=1,
        push_toward=push_toward,
        l=l,
        kappa=player_config["kappa"],
        gamma=player_config["gamma"]
    )
    try:
        optimal_node, optimal_message, max_influence = player.optimize_strategy_single_agent(G, C)
        _, A_updated, claimed_nodes, influence_history = simulate_diffusion(player, G, optimal_node, optimal_message, C)
        return {
            "Simulation": sim_idx,
            "Topic": topic_index,
            "Push Direction": push_toward,
            "Initial Node": optimal_node,
            "Optimal Message": optimal_message,
            "Total Influence": max_influence,
            "Steps": len(influence_history),
            "Updated Opinons": A_updated
        }
    except Exception as e:
        print(f"Simulation {sim_idx} Topic {topic_index} failed: {e}")
        return None


In [10]:
#intialization for Simulations
N_SIMULATIONS = 50
TOPICS = [0, 1, 2]
PUSH = -1  
n_jobs = -1  #all available CPU cores

# Define shared config
player_config = {
    "theta_low": 0.3,
    "theta_high": 0.7,
    "delta": 0.4,
    "beta": 1,
    "kappa": 0.3,
    "gamma": 0.2
}

#job list for parallel processing
job_list = [(sim, topic, PUSH, G, C_stochastic, A_initial, l, player_config)
            for sim in range(N_SIMULATIONS) for topic in TOPICS]

#Run in parallel
results = Parallel(n_jobs=n_jobs)(
    delayed(run_single_simulation)(*job) for job in job_list
)

#Filter for failed runs
results_clean = [r for r in results if r is not None]

# ave to DataFrame
df_results = pd.DataFrame(results_clean)
df_results.to_csv("[DIRECTORY]/parallel_simulation_resultsParallelto-1.csv", index=False)
print("Saved results to parallel_simulation_resultsParallelto-1.csv")


Saved results to monteto-1.csv
